In [59]:
import sqlite3
import csv
# Establish a connection to the database (create a new database if it doesn't exist)
conn = sqlite3.connect('cars.db')
# Create a cursor object to execute SQL queries
cursor = conn.cursor()
# Create the Makes table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Makes (
        id INTEGER PRIMARY KEY,
        name VARCHAR(30) NOT NULL
    )
''')
# Create the Model table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Models (
        id INTEGER PRIMARY KEY,
        make_id INTEGER,
        name VARCHAR(30) NOT NULL,
        FOREIGN KEY (make_id) REFERENCES Makes(id)
    )
''')
# Create the Transmission table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Transmissions (
        id INTEGER PRIMARY KEY,
        type VARCHAR(30) NOT NULL
    )
''')
# Create the Drive_Type table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Drive_Types (
        id INTEGER PRIMARY KEY,
        type VARCHAR(30) NOT NULL
    )
''')
# Create the Listing table
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Listings (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        model_id INTEGER,
        year INTEGER,
        mileage INTEGER,
        transmission_id INTEGER,
        fuel_type VARCHAR(30),
        engine VARCHAR(30),
        body_type VARCHAR(30),
        vehicle_title VARCHAR(100),
        page_title VARCHAR(100),
        price DECIMAL,
        sold_date DATETIME,
        url VARCHAR(255),
        FOREIGN KEY (model_id) REFERENCES Models(id),
        FOREIGN KEY (transmission_id) REFERENCES Transmissions(id)
    )
''')
def get_or_insert(table, column, value):
    cursor.execute(f'SELECT id FROM {table} WHERE {column} = ?', (value,))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        cursor.execute(f'INSERT INTO {table} ({column}) VALUES (?)', (value,))
        return cursor.lastrowid
# Read data from the CSV file (assuming the file name is 'data.csv')
with open('data/compiled_data.csv', 'r') as file:
    csv_data = csv.DictReader(file)
    # Iterate over each row in the CSV file
    for row in csv_data:
        make_id = get_or_insert("Makes", "name", row["make"])
        model_id = get_or_insert("Models", "name", row["model"])
        cursor.execute('INSERT INTO Models (make_id, name) VALUES (?, ?)', (make_id, row["model"]))
        transmission_id = get_or_insert("Transmissions", "type", row["transmission"])
        drive_type_id = get_or_insert("Drive_Types", "type", row["drive_type"])
        # Insert the data into the Listing table
        cursor.execute('''
            INSERT INTO Listings (model_id, year, mileage, transmission_id, fuel_type, engine, body_type, vehicle_title, page_title, price, sold_date, url)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''', (model_id, row["year"],
row["mileage"], transmission_id, row["fuel_type"], row["engine"], row["body_type"], row["vehicle_title"], row["page_title"], row["price"], row["sold_date"], row["url"]))
conn.commit()
conn.close()
print("Database has been populated successfully.")

Database has been populated successfully.


In [60]:
# Establish a connection to the database
conn = sqlite3.connect('cars.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Query to retrieve table names
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")

# Fetch all the table names
tables = cursor.fetchall()

# Print the table names
for table in tables:
    print(table[0])


Cars
Makes
Model
Transmission
Drive_Type
Listing
Models
Transmissions
Drive_Types
Listings
sqlite_sequence


In [61]:
import sqlite3

# Establish a connection to the database
conn = sqlite3.connect('cars.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Query to retrieve the contents of a table
table_name = 'Listing'  # Replace with the actual table name
cursor.execute(f"SELECT * FROM {table_name}")

# Fetch all the rows from the table
rows = cursor.fetchall()

# Print the column names
column_names = [description[0] for description in cursor.description]
print(column_names)

# Print the contents of the table
for row in rows:
    print(row)

['id', 'model', 'year', 'mileage', 'transmission', 'url', 'sold_date', 'price', 'page_title', 'vehicle_title', 'drive_type']


In [62]:
import sqlite3
# Establish a connection to the database
conn = sqlite3.connect('cars.db')
# Create a cursor object to execute SQL queries
cursor = conn.cursor()
# Get the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
# Iterate through each table and display columns and first 5 rows
for table in tables:
    table_name = table[0]
    print(f"Table '{table_name}':")
    
    # Query to get column information
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = cursor.fetchall()
    
    # Display column names
    col_names = [column[1] for column in columns]
    print(f"  {' | '.join(col_names)}")
    
    # Query to get the first 5 rows
    cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
    rows = cursor.fetchall()
    
    # Display rows
    for row in rows:
        print(f"  {' | '.join(map(str, row))}")
    
    print("")  # Add an empty line between tables
# Close the connection
conn.commit()
conn.close()
print("Database connection closed")

Table 'Cars':
  Title | Price | Sold_Date | Item_URL

Table 'Makes':
  id | name
  1 | Cadillac
  2 | Toyota
  3 | Jaguar
  4 | Nissan
  5 | Hyundai

Table 'Model':
  id | make | name

Table 'Transmission':
  id | type

Table 'Drive_Type':
  id | type

Table 'Listing':
  id | model | year | mileage | transmission | url | sold_date | price | page_title | vehicle_title | drive_type

Table 'Models':
  id | make_id | name
  1 | None | CT6
  2 | 1 | CT6
  3 | None | Land Cruiser
  4 | 2 | Land Cruiser
  5 | None | F-Type

Table 'Transmissions':
  id | type
  1 | Automatic
  2 | Manual
  3 | 8 Speed Tiptronic Automatic
  4 | Single-Speed Fixed Gear
  5 | 7 speed automatic

Table 'Drive_Types':
  id | type
  1 | AWD
  2 | 4WD
  3 | FWD
  4 | RWD
  5 | 4X2

Table 'Listings':
  id | model_id | year | mileage | transmission_id | fuel_type | engine | body_type | vehicle_title | page_title | price | sold_date | url
  1 | 1 | 2019 | 51900 | 1 | Gasoline | 3.6L | Sedan | Rebuilt, Rebuildable & Recon